In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization
from keras.preprocessing.sequence import TimeseriesGenerator

%matplotlib inline  




C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\

In [2]:
#yf.download('GE',start='2019-06-01', interval='1d',  end='2021-06-01',progress=False)
df = yf.download('GE',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False)

df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
df["Date"] = df.index
df.head()
 
#data.plot(figsize=(10,10))

,Close,Adj Close,Date
Date,,,
2016-06-01,28.951923,26.341440,2016-06-01
2016-06-02,28.894232,26.288946,2016-06-02
2016-06-03,28.788462,26.192719,2016-06-03
2016-06-06,28.961538,26.350182,2016-06-06
2016-06-07,28.980768,26.367683,2016-06-07


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1287 entries, 2016-06-01 to 2021-07-12
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Close      1287 non-null   float64       
 1   Adj Close  1287 non-null   float64       
 2   Date       1287 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 40.2 KB


In [4]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

1029
258


In [5]:
look_back = 30

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [6]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 100
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


Epoch 1/100
50/50 [==============================] - 1s 17ms/step - loss: 116.9245
Epoch 2/100
50/50 [==============================] - 0s 9ms/step - loss: 2.0314
Epoch 3/100
50/50 [==============================] - 0s 9ms/step - loss: 1.1456
Epoch 4/100
50/50 [==============================] - 0s 9ms/step - loss: 0.7814
Epoch 5/100
50/50 [==============================] - 0s 9ms/step - loss: 0.6274
Epoch 6/100
50/50 [==============================] - 0s 9ms/step - loss: 0.5874
Epoch 7/100
50/50 [==============================] - 0s 9ms/step - loss: 0.5824
Epoch 8/100
50/50 [==============================] - 0s 9ms/step - loss: 0.5057
Epoch 9/100
50/50 [==============================] - 0s 9ms/step - loss: 0.5514
Epoch 10/100
50/50 [==============================] - 0s 9ms/step - loss: 0.4118
Epoch 11/100
50/50 [==============================] - 0s 8ms/step - loss: 0.3279
Epoch 12/100
50/50 [==============================] - 0s 9ms/step - loss: 0.4813
Epoch 13/100
50/50 [=============

In [7]:
#Test
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [8]:
# close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates) 

[Timestamp('2021-07-12 00:00:00', freq='D'), Timestamp('2021-07-13 00:00:00', freq='D'), Timestamp('2021-07-14 00:00:00', freq='D'), Timestamp('2021-07-15 00:00:00', freq='D'), Timestamp('2021-07-16 00:00:00', freq='D'), Timestamp('2021-07-17 00:00:00', freq='D'), Timestamp('2021-07-18 00:00:00', freq='D'), Timestamp('2021-07-19 00:00:00', freq='D'), Timestamp('2021-07-20 00:00:00', freq='D'), Timestamp('2021-07-21 00:00:00', freq='D'), Timestamp('2021-07-22 00:00:00', freq='D'), Timestamp('2021-07-23 00:00:00', freq='D'), Timestamp('2021-07-24 00:00:00', freq='D'), Timestamp('2021-07-25 00:00:00', freq='D'), Timestamp('2021-07-26 00:00:00', freq='D'), Timestamp('2021-07-27 00:00:00', freq='D'), Timestamp('2021-07-28 00:00:00', freq='D'), Timestamp('2021-07-29 00:00:00', freq='D'), Timestamp('2021-07-30 00:00:00', freq='D'), Timestamp('2021-07-31 00:00:00', freq='D'), Timestamp('2021-08-01 00:00:00', freq='D'), Timestamp('2021-08-02 00:00:00', freq='D'), Timestamp('2021-08-03 00:00:00'

In [9]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
pfig = go.Figure(data=[trace1, trace2], layout=layout)
pfig.show()